## Trabajo en información conjunta

In [2]:
# Dependencias básicas
import pandas as pd
import numpy as np

# Dependencias de visualización
from matplotlib import pyplot as plt
import seaborn as sns
import altair as alt

# Dependencias de machine learning
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer


import plotly.express as px
import plotly.graph_objects as go
from ipywidgets import interact

In [5]:
# Importación de datasets
inmigrantes_data = pd.read_csv("../13 - Exports (preprocesamiento)/top_inmigracion_2008_2022.csv")
democracia = pd.read_csv("../13 - Exports (preprocesamiento)/indices_democracia.csv")
regiones = pd.read_csv("../13 - Exports (preprocesamiento)/continentes_regiones.csv")
libertad = pd.read_csv("../13 - Exports (preprocesamiento)/libertad.csv")
conflictos = pd.read_csv("../13 - Exports (preprocesamiento)/muertes_conflictos_armados.csv")
nacional = pd.read_csv("../13 - Exports (preprocesamiento)/nacional.csv")
regimen = pd.read_csv("../13 - Exports (preprocesamiento)/regimen_politico.csv")
homicidios = pd.read_csv("../13 - Exports (preprocesamiento)/tasa_homicidios.csv")
turismo = pd.read_csv("../13 - Exports (preprocesamiento)/turismo.csv")



In [6]:
# Hacer una copia de datos de inmigrantes para los merge
inmigrantes = inmigrantes_data.copy()
inmigrantes

,Year,Nationality code,Sex,Age group,Immigrant count
0,2008,DZA,Both,0 - 14,759
1,2008,PER,Males,35 - 44,2938
2,2008,PER,Males,45 - 54,1128
3,2008,PER,Males,55 - 64,265
4,2008,PER,Males,65+,156
...,...,...,...,...,...
9043,2022,VEN,Males,55 - 64,1606
9044,2022,VEN,Females,55 - 64,3913
9045,2022,VEN,Both,65+,4665
9046,2022,VEN,Males,65+,1407


In [3]:
democracia.head()

,Nationality code,Year,Liberal democracy index,Deliberative democracy index
0,DZA,2008,0.164,0.249
1,DZA,2009,0.163,0.246
2,DZA,2010,0.162,0.250
3,DZA,2011,0.162,0.250
4,DZA,2012,0.165,0.252


In [5]:
regiones.head()

,Continent,Sub-region,Nationality code
0,Europe,European Union,BGR
1,Europe,European Union,FRA
2,Europe,European Union,ITA
3,Europe,European Union,PRT
4,Europe,European Union,DEU


**Para "libertad" el problema es que faltaba pivotear los indicadores a columnas, sino no se podría hacer el join.**

In [6]:
libertad.head()

,Country code,Indicator,Year,Value
0,ARG,Absence of Corruption,2008,0.51
1,ARG,Access to Justice,2008,0.53
2,ARG,Basic Welfare,2008,0.62
3,ARG,Civic Engagement,2008,0.80
4,ARG,Civil Liberties,2008,0.80


In [7]:
conflictos.head()

,Nationality code,Year,One-sided violence_deaths,Non-state_deaths,Intrastate_deaths,Interstate_deaths
0,DZA,2008,0,0,345,0
1,DZA,2009,36,0,508,0
2,DZA,2010,0,0,236,0
3,DZA,2011,0,0,261,0
4,DZA,2012,0,0,258,0


In [4]:
nacional.head()

,Nacionalidad,Sex,Year,Total,Country,Country Code
0,Alemania,Both,2022,116122,Germany,DEU
1,Alemania,Both,2021,109556,Germany,DEU
2,Alemania,Both,2020,111937,Germany,DEU
3,Alemania,Both,2019,111911,Germany,DEU
4,Alemania,Both,2018,111495,Germany,DEU


In [ ]:
nacional

In [9]:
regimen.head()

,Nationality code,Year,Political regime
0,DZA,2008,3
1,DZA,2009,3
2,DZA,2010,3
3,DZA,2011,3
4,DZA,2012,3


In [10]:
homicidios.head()

,Year,Nationality code,Homicide Rate
0,2008,ARG,5.89
1,2008,BGR,2.24
2,2008,BRA,26.02
3,2008,CHN,1.11
4,2008,COL,36.84


In [11]:
turismo.head()

,Year,Total
0,2000,34380000
1,2001,35330000
2,2002,34950000
3,2003,36920000
4,2004,38520000


In [30]:
libertad = libertad.pivot_table(
    index=['Country code', 'Year'],
    columns='Indicator',             
    values='Value'                   
).reset_index()

In [31]:
libertad

Indicator,Country code,Year,Absence of Corruption,Access to Justice,Access to justice for men,Access to justice for women,Association/ assembly rights,Barriers to parties,Basic Welfare,Basic administration,...,Social capital,Social class equality in respect for civil liberties,Social group equality in respect for civil liberties,Subnational elections free and fair,Transparent laws with predictable enforcement,Voter turnout,Women's economic rights,Women's political rights,Workers' rights,violations of user rights
0,ARG,2008,0.51,0.53,0.65,0.62,1.00,1.0,0.62,0.78,...,0.75,0.67,0.66,0.81,0.54,0.72,1.00,0.08,0.43,NaN
1,ARG,2009,0.51,0.53,0.65,0.62,1.00,1.0,0.64,0.78,...,0.75,0.67,0.66,0.81,0.54,0.71,1.00,0.08,0.50,NaN
2,ARG,2010,0.51,0.53,0.65,0.62,1.00,1.0,0.64,0.78,...,0.75,0.67,0.66,0.81,0.54,0.71,1.00,0.08,0.57,NaN
3,ARG,2011,0.51,0.53,0.65,0.62,1.00,1.0,0.64,0.78,...,0.75,0.67,0.66,0.81,0.55,0.77,1.00,0.08,0.64,NaN
4,ARG,2012,0.49,0.54,0.65,0.62,1.00,1.0,0.64,0.78,...,0.75,0.67,0.66,0.81,0.55,0.77,1.00,0.08,0.43,0.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,VEN,2018,0.06,0.22,0.30,0.36,0.33,0.5,0.54,0.44,...,0.25,0.44,0.57,0.31,0.17,0.43,0.67,0.09,0.21,0.35
386,VEN,2019,0.06,0.22,0.36,0.36,0.33,0.5,0.54,0.44,...,0.25,0.43,0.55,0.31,0.17,0.43,0.67,0.10,0.21,0.32
387,VEN,2020,0.00,0.22,0.31,0.40,0.22,0.5,0.53,0.22,...,0.12,0.39,0.68,0.31,0.18,0.30,NaN,0.12,NaN,0.27
388,VEN,2021,0.00,0.24,0.37,0.41,0.22,0.5,0.53,0.22,...,0.12,0.35,0.74,0.40,0.26,0.30,0.67,0.13,NaN,0.27


In [ ]:
democracia.rename(columns={"Nationality code": "Country code"}, inplace = True)

In [33]:
lib_demo = pd.merge(libertad, democracia, on = ["Country code", "Year"], how = "left")
lib_demo

,Country code,Year,Absence of Corruption,Access to Justice,Access to justice for men,Access to justice for women,Association/ assembly rights,Barriers to parties,Basic Welfare,Basic administration,...,Social group equality in respect for civil liberties,Subnational elections free and fair,Transparent laws with predictable enforcement,Voter turnout,Women's economic rights,Women's political rights,Workers' rights,violations of user rights,Liberal democracy index,Deliberative democracy index
0,ARG,2008,0.51,0.53,0.65,0.62,1.00,1.0,0.62,0.78,...,0.66,0.81,0.54,0.72,1.00,0.08,0.43,NaN,0.605,0.567
1,ARG,2009,0.51,0.53,0.65,0.62,1.00,1.0,0.64,0.78,...,0.66,0.81,0.54,0.71,1.00,0.08,0.50,NaN,0.601,0.573
2,ARG,2010,0.51,0.53,0.65,0.62,1.00,1.0,0.64,0.78,...,0.66,0.81,0.54,0.71,1.00,0.08,0.57,NaN,0.589,0.556
3,ARG,2011,0.51,0.53,0.65,0.62,1.00,1.0,0.64,0.78,...,0.66,0.81,0.55,0.77,1.00,0.08,0.64,NaN,0.585,0.557
4,ARG,2012,0.49,0.54,0.65,0.62,1.00,1.0,0.64,0.78,...,0.66,0.81,0.55,0.77,1.00,0.08,0.43,0.86,0.587,0.561
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
385,VEN,2018,0.06,0.22,0.30,0.36,0.33,0.5,0.54,0.44,...,0.57,0.31,0.17,0.43,0.67,0.09,0.21,0.35,0.060,0.040
386,VEN,2019,0.06,0.22,0.36,0.36,0.33,0.5,0.54,0.44,...,0.55,0.31,0.17,0.43,0.67,0.10,0.21,0.32,0.059,0.041
387,VEN,2020,0.00,0.22,0.31,0.40,0.22,0.5,0.53,0.22,...,0.68,0.31,0.18,0.30,NaN,0.12,NaN,0.27,0.057,0.040
388,VEN,2021,0.00,0.24,0.37,0.41,0.22,0.5,0.53,0.22,...,0.74,0.40,0.26,0.30,0.67,0.13,NaN,0.27,0.060,0.045
